# Пошаговое руководство по созданию жидкой нейронной сети LTC с нуля
Павел Наказненко, 2024

[Liquid Time-Constant Networks on Arxiv](https://arxiv.org/abs/2006.04439)

Благодарность: Это руководство в значительной степени основано на [реализации LTCCell](https://github.com/mlech26l/ncps/blob/master/ncps/torch/ltc_cell.py), спасибо авторам LNN.

[Канал Telegram: ToShoSeti](https://t.me/toshoseti)

## Отказ от ответственности
Это руководство является попыткой ознакомиться с LNN и понять, как они работают. Это не идеальная реализация.
Для меня это сложная тема, поэтому возможны ошибки и недопонимания. Пожалуйста, будьте внимательны.

## Imports

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim

## Реализация класса RandomWiring
Класс RandomWiring отвечает за определение архитектуры соединений между нейронами. Он инициализирует случайные матрицы смежности для соединений нейронов и сенсорных входов. Они используются в качестве произвольной разреженной матрицы позже в LIFNeuralLayer.

In [ ]:
class RandomWiring:
    def __init__(self, input_dim, output_dim, neuron_count):
        self.input_dim = input_dim  # Количество входных признаков
        self.output_dim = output_dim  # Количество выходных признаков
        self.neuron_count = neuron_count  # Количество нейронов в слое
        self.adjacency_matrix = np.random.uniform(0, 1, (neuron_count, neuron_count))  # Матрица смежности для соединений между нейронами
        self.sensory_adjacency_matrix = np.random.uniform(0, 1, (input_dim, neuron_count))  # Матрица смежности для сенсорных входов к нейронам

    def erev_initializer(self):
        return np.random.uniform(-0.2, 0.2, (self.neuron_count, self.neuron_count))  # Инициализация потенциалов разворота для соединений нейронов

    def sensory_erev_initializer(self):
        return np.random.uniform(-0.2, 0.2, (self.input_dim, self.neuron_count))  # Инициализация потенциалов разворота для сенсорных входов

## Реализация класса LIFNeuronLayer
Класс LIFNeuronLayer моделирует поведение слоя нейронов типа Leaky Integrate-and-Fire. Он инициализирует параметры нейронов и определяет прямой проход для вычисления состояний нейронов. Динамика LIF описывается с помощью ОДУ (обыкновенных дифференциальных уравнений), и во время прямого прохода мы вычисляем состояния, используя метод Эйлера Explicit, описанный в оригинальной статье.

In [ ]:
class LIFNeuronLayer(nn.Module):
    def __init__(self, wiring, ode_unfolds=12, epsilon=1e-8):
        super(LIFNeuronLayer, self).__init__()
        self.wiring = wiring  # Объект Wiring, содержащий информацию о соединениях
        self.ode_unfolds = ode_unfolds  # Количество итераций решателя ОДУ
        self.epsilon = epsilon  # Малое значение, чтобы избежать деления на ноль
        self.softplus = nn.Softplus()  # Функция активации Softplus

        # Диапазоны инициализации для параметров
        GLEAK_MIN, GLEAK_MAX = 0.001, 1.0
        VLEAK_MIN, VLEAK_MAX = -0.2, 0.2
        CM_MIN, CM_MAX = 0.4, 0.6
        W_MIN, W_MAX = 0.001, 1.0
        SIGMA_MIN, SIGMA_MAX = 3, 8
        MU_MIN, MU_MAX = 0.3, 0.8
        SENSORY_W_MIN, SENSORY_W_MAX = 0.001, 1.0
        SENSORY_SIGMA_MIN, SENSORY_SIGMA_MAX = 3, 8
        SENSORY_MU_MIN, SENSORY_MU_MAX = 0.3, 0.8

        # Инициализация параметров нейронов случайными значениями в указанных диапазонах
        self.gleak = nn.Parameter(torch.rand(wiring.neuron_count) * (GLEAK_MAX - GLEAK_MIN) + GLEAK_MIN)
        self.vleak = nn.Parameter(torch.rand(wiring.neuron_count) * (VLEAK_MAX - VLEAK_MIN) + VLEAK_MIN)
        self.cm = nn.Parameter(torch.rand(wiring.neuron_count) * (CM_MAX - CM_MIN) + CM_MIN)
        self.w = nn.Parameter(torch.rand(wiring.neuron_count, wiring.neuron_count) * (W_MAX - W_MIN) + W_MIN)
        self.sigma = nn.Parameter(torch.rand(wiring.neuron_count, wiring.neuron_count) * (SIGMA_MAX - SIGMA_MIN) + SIGMA_MIN)
        self.mu = nn.Parameter(torch.rand(wiring.neuron_count, wiring.neuron_count) * (MU_MAX - MU_MIN) + MU_MIN)
        self.erev = nn.Parameter(torch.Tensor(wiring.erev_initializer()))
        
        # Инициализация сенсорных параметров случайными значениями в указанных диапазонах
        self.sensory_w = nn.Parameter(torch.rand(wiring.input_dim, wiring.neuron_count) * (SENSORY_W_MAX - SENSORY_W_MIN) + SENSORY_W_MIN)
        self.sensory_sigma = nn.Parameter(torch.rand(wiring.input_dim, wiring.neuron_count) * (SENSORY_SIGMA_MAX - SENSORY_SIGMA_MIN) + SENSORY_SIGMA_MIN)
        self.sensory_mu = nn.Parameter(torch.rand(wiring.input_dim, wiring.neuron_count) * (SENSORY_MU_MAX - SENSORY_MU_MIN) + SENSORY_MU_MIN)
        self.sensory_erev = nn.Parameter(torch.Tensor(wiring.sensory_erev_initializer()))

        # Маски разреженности (фиксированные, необучаемые) на основе матриц смежности соединений
        self.sparsity_mask = torch.Tensor(np.abs(wiring.adjacency_matrix))
        self.sensory_sparsity_mask = torch.Tensor(np.abs(wiring.sensory_adjacency_matrix))

    def forward(self, inputs, state, elapsed_time=1.0):
        return self.ode_solver(inputs, state, elapsed_time)

    def ode_solver(self, inputs, state, elapsed_time):
        v_pre = state  # Предыдущее состояние (напряжение)

        # Предварительный расчет эффектов от сенсорных нейронов
        sensory_activation = self.softplus(self.sensory_w) * self.sigmoid(inputs, self.sensory_mu, self.sensory_sigma)
        sensory_activation = sensory_activation * self.sensory_sparsity_mask
        sensory_reversal_activation = sensory_activation * self.sensory_erev

        # Расчет числителя и знаменателя для сенсорных входов
        w_numerator_sensory = torch.sum(sensory_reversal_activation, dim=1)
        w_denominator_sensory = torch.sum(sensory_activation, dim=1)

        # Расчет мембранной емкости во времени
        cm_t = self.softplus(self.cm) / (elapsed_time / self.ode_unfolds)

        # Инициализация весов для связей между нейронами
        w_param = self.softplus(self.w)
        for _ in range(self.ode_unfolds):
            # Активация на основе предыдущего состояния
            w_activation = w_param * self.sigmoid(v_pre, self.mu, self.sigma)
            w_activation = w_activation * self.sparsity_mask
            reversal_activation = w_activation * self.erev

            # Расчет числителя и знаменателя для связей между нейронами
            w_numerator = torch.sum(reversal_activation, dim=1) + w_numerator_sensory
            w_denominator = torch.sum(w_activation, dim=1) + w_denominator_sensory

            # Расчет проводимости утечки и напряжения
            gleak = self.softplus(self.gleak)
            numerator = cm_t * v_pre + gleak * self.vleak + w_numerator
            denominator = cm_t + gleak + w_denominator

            # Обновление состояния (напряжения)
            v_pre = numerator / (denominator + self.epsilon)

        return v_pre

    def sigmoid(self, v_pre, mu, sigma):
        v_pre = torch.unsqueeze(v_pre, -1)  # Расширение для соответствия размерам
        activation = sigma * (v_pre - mu)  # Применение сигма и среднего смещения
        return torch.sigmoid(activation)  # Применение сигмоидной активации

### Понимание нейронного слоя LIF

Нейрон Leaky Integrate-and-Fire (LIF) — это простая и широко используемая модель в вычислительной нейробиологии. Он моделирует, как биологические нейроны интегрируют поступающие сигналы и срабатывают при достижении порога. Модель LIF отражает основные характеристики реальных нейронов и является эффективной в вычислительном плане.

#### Математическая формулировка LIF-нейронов

Нейрон LIF можно описать следующим дифференциальным уравнением:

$$\tau_m \frac{dV(t)}{dt} = -V(t) + R_m I(t)$$

Где:
- $V(t)$ — мембранный потенциал в момент времени $t$.
- $\tau_m$ – постоянная времени мембраны.
- $R_m$ – сопротивление мембраны.
- $I(t)$ — входной ток в момент времени $t$.

Когда мембранный потенциал $V(t)$ достигает определенного порога $V_{\text{th}}$, нейрон запускает потенциал действия (или спайк), и $V(t)$ сбрасывается до более низкого значения, часто $V_{\text{reset}}$.

#### Реализация в классе LIFNeuronLayer

Класс `LIFNeuronLayer` в нашей реализации имитирует слой LIF-нейронов с конкретными параметрами мембранного потенциала, проводимости и входных весов. 
##### Инициализация

Класс инициализирует несколько параметров, управляющих поведением нейронов, в том числе:
- **gleak**: проводимость утечки.
- **vleak**: напряжение утечки.
- **см**: емкость мембраны.
- **w, sigma, mu**: параметры весов и функций активации.
- **erev**: Реверсивные потенциалы нейронных связей.
- **sensory_w, sensory_sigma, sensory_mu, sensory_erev**: Параметры сенсорных входов.

#### Отмена активации

- **Реверсивная активация**: представляет собой влияние синаптических реверсивных потенциалов на мембранный потенциал. В биологических нейронах реверсивный потенциал — это напряжение, при котором чистый поток определенного иона через мембрану равен нулю. Включение в модель реверсивных потенциалов помогает моделировать реалистичные синаптические взаимодействия.
- **Расчет**: включает умножение активации на матрицу реверсивного потенциала, которая влияет на обновление состояния нейрона во время прямого прохода. Этот механизм помогает интегрировать эффекты возбуждающих и тормозных синапсов.

##### Forward проход и солвер ОДУ

Прямой проход «LIFNeuronLayer» включает решение дифференциального уравнения, которое управляет динамикой мембранного потенциала. Это делается с использованием итеративного подхода с несколькими развертываниями ОДУ.

Упрощенно:
1. **Предварительный расчет сенсорных эффектов**: рассчитываем сенсорную активацию и обратную активацию на основе входных данных.
2. **Цикл солвера ОДУ**: итеративно обновляем состояния нейронов с помощью солвера ОДУ. Это включает в себя вычисление активаций, применение масок разреженности и обновление состояний напряжения.

##### Сигмоидная функция

Классическая сигмоидная функция масштабирует входные данные в диапазоне от 0 до 1, что важно для активации нейронов. Она используется как для нейронных связей, так и для сенсорных входов.
Измененная сигмоидная функция в классе LIFNeuronLayer используется для включения параметров mu (среднее значение) и сигма (масштаб). Эта измененная реализация обеспечивает более гибкое и контролируемое поведение активации по сравнению с классическим torch.sigmoid. 

1. **Параметр «mu» (средний сдвиг)**:
    - Параметр `mu` сдвигает входное напряжение (`v_pre`). Этот сдвиг позволяет центрировать функцию активации вокруг различных средних значений.
    - Математически `v_pre - mu` сдвигает входные данные так, что средняя точка (где сигмоида равна 0,5) не обязательно равна нулю, а `mu`.

2. **Параметр `sigma` (Масштаб)**:
    - Параметр `sigma` масштабирует входное напряжение (`v_pre`). Это масштабирование контролирует крутизну сигмовидной функции.
    - Математически `sigma * (v_pre - mu)` регулирует наклон сигмовидной кривой. Большая «сигма» делает сигмовидную круче, а меньшая «сигма» делает ее более плавной.

Измененная сигмовидная функция используется для обеспечения большей гибкости в динамике активации нейронной сети:

1. **Биологический реализм**:
    - Биологические нейроны не имеют фиксированного порога активации или кривой ответа. Кривая отклика может смещаться и масштабироваться в зависимости от различных факторов.
    - Объединив «мю» и «сигму», мы можем смоделировать эту изменчивость, сделав модель более биологически правдоподобной.

2. **Улучшенное обучение**:
    - Нейронные сети могут извлечь выгоду из функций адаптивной активации. Разным слоям или нейронам может потребоваться разное поведение активации для эффективного изучения сложных паттернов.
    - Параметры «mu» и «sigma» можно подобрать во время обучения, что позволяет сети динамически настраивать свои функции активации.
3. **Расширенный контроль**:
    - В некоторых случаях контроль над средним значением и масштабом функции активации может улучшить способность сети обрабатывать различные входные диапазоны и распределения.
    - Такая настройка может привести к лучшей сходимости и производительности в определенных задачах, особенно в тех, которые связаны с разнообразными и сложными входными сигналами.

#### Зачем нужны маски случайной смежности и разреженности?

- **Матрица случайной смежности**: эта матрица определяет связи между нейронами случайным образом, моделируя сложные и нерегулярные связи, обнаруженные в биологических нейронных сетях. Это вносит в сеть изменчивость и сложность, что может помочь в изучении более разнообразных моделей.
- **Маска разреженности**: эта маска гарантирует, что активны только определенные соединения, обеспечивая разреженность сети. Разреженные соединения имитируют эффективную проводку мозга, где не каждый нейрон связан с каждым другим нейроном, что снижает вычислительную нагрузку и предотвращает переобучение.

## Реализация класса LTCCell
Класс LTCCell представляет одну ячейку в Liquid Time-Constant Recurrent Neural Network. Он использует LIFNeuronLayer для обновления состояний нейронов.

Примечание: мы предполагаем, что количество нейронов всегда больше выходного измерения, поэтому мы просто обрезаем их до выходных значений. Если это не так, можно использовать дополнительный полносвязный слой для проецирования скрытых состояний на выходные данные.

In [ ]:
class LTCCell(nn.Module):
     def __init__(self, wiring, in_features=None, ode_unfolds=6, epsilon=1e-8):
         super(LTCCell, self).__init__()
         self.wiring = wiring # Объект Wiring
         self.ode_unfolds = ode_unfolds # Количество итераций решателя ОДУ
         self.epsilon = epsilon # Небольшое значение, чтобы избежать деления на ноль

         self.neuron = LIFNeuronLayer(wiring, ode_unfolds, epsilon) # Инициализируем LIFNeuron с заданными соединениями

     def forward(self, inputs,states, elapsed_time=1.0):
         next_state = self.neuron(inputs,states,elapsed_time) # Вычисляем следующее состояние, используя модель нейрона
         outputs = next_state[:, :self.wiring.output_dim] # Сопоставляем состояние с выходными измерениями
         return outputs, next_state

## Реализация класса LTCRNN
Класс LTCRNN создает рекуррентную нейронную сеть, используя несколько экземпляров LTCCell. Он обрабатывает последовательности входных данных для получения последовательностей выходных данных.

Примечание: В этом руководстве предполагается, что LTCRNN будет вызываться один раз для всей последовательности, поэтому здесь происходит инициализация состояния нулевым значением при каждом вызове forward.

In [ ]:
class LTCRNN(nn.Module):
    def __init__(self, wiring, input_dim, hidden_dim, output_dim):
        super(LTCRNN, self).__init__()
        self.cell = LTCCell(wiring, in_features=input_dim)  # Инициализация LTCCell с wiring и размером входных данных
        self.hidden_dim = hidden_dim  # Количество скрытых нейронов
        self.output_dim = output_dim  # Количество выходных нейронов
        
    def forward(self, inputs):
        batch_size, seq_len, _ = inputs.size()  # Получение размера батча и длины последовательности из размеров входных данных
        
        states = torch.zeros(batch_size, self.hidden_dim)  # Инициализация скрытых состояний нулями
            
        outputs = []  # Список для хранения выходных данных для каждого временного шага

        for t in range(seq_len):
            output, states = self.cell(inputs[:, t, :], states)  # Вычисление выходных данных и следующего состояния для каждого временного шага
            outputs.append(output)  # Добавление выходных данных в список

        result = torch.stack(outputs, dim=1)  # стэкаем выходные данные вдоль размерности последовательности
        return result

## Генерация спиральных данных
Мы сгенерируем набор данных спиральных траекторий для обучения и оценки нашей модели. Функция generate_spiral_data создает синтетические точки данных, образующие спиральный узор.

In [ ]:
def generate_spiral_data(num_points, num_turns, noise = 2):
    theta = np.linspace(0, num_turns * 2 * np.pi, num_points)
    z = np.linspace(0, 1, num_points)
    r = z
    x = r * np.sin(theta) + noise * np.random.randn(*theta.shape) / num_points
    y = r * np.cos(theta) + noise * np.random.randn(*theta.shape) / num_points
    return np.stack([x, y], axis=1)

## Обучение модели
Далее мы устанавливаем гиперпараметры и готовим данные для обучения. Мы определяем цикл обучения для обучения модели с использованием сгенерированных спиральных данных.

In [ ]:
# Гиперпараметры
input_dim = 2 # Количество входных размерностей
hidden_dim = 8 # Количество скрытых размерностей (количество нейронов в LIFNeuralLayer)
output_dim = 2 # Количество выходных размерностей
num_points = 500 # Количество точек спирали в наборе данных
num_turns = 3 # Количество витков спирали
learning_rate = 0.005
num_epochs = 2000
seq_len = 3 # Максимальная длина последовательности выборки
batch_size = 32

# Генерация данных
data = generate_spiral_data(num_points, num_turns)
all_inputs = data[:-1, :]
all_targets = data[1:, :]

# Подготовка входных и целевых последовательностей
trajectory_count = max(1, len(all_inputs) - seq_len)
train_inputs = [torch.FloatTensor(all_inputs[i:i + seq_len]) for i in range(trajectory_count)]
train_targets = [torch.FloatTensor(all_targets[i:i + seq_len]) for i in range(trajectory_count)]

# Перемешивание и разделение данных для обучения
random_train_indices = np.arange(len(train_inputs))
np.random.shuffle(random_train_indices)
train_split_index = int(len(random_train_indices) * 0.8)
random_train_indices = random_train_indices[:train_split_index]

# Функция для создания батчей
def create_batches(data_list, batch_size):
    return [data_list[i:i + batch_size] for i in range(0, len(data_list), batch_size)]

# Создание батчей входных и целевых данных
train_input_batches = create_batches(train_inputs, batch_size)
train_target_batches = create_batches(train_targets, batch_size)

# Инициализация модели
wiring = RandomWiring(input_dim, output_dim, hidden_dim)
model = LTCRNN(wiring, input_dim, hidden_dim, output_dim)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Цикл обучения
for epoch in range(num_epochs):
    model.train()    
    total_loss = 0

    # Итерация по партиям
    for x, y_target in zip(train_input_batches, train_target_batches):
        optimizer.zero_grad()
        x = torch.stack(x)  # Складывание партии последовательностей
        y_target = torch.stack(y_target)  # Складывание партии целей
        outputs = model(x)  # Прямой проход через модель
        loss = criterion(outputs, y_target)  # Вычисление потерь

        # Накопление общих потерь и выполнение обратного прохода и шага оптимизации
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    # Вывод потерь каждые 100 эпох и построение прогнозов
    if (epoch + 1) % 100 == 0:
        # Прогнозирование и построение графика
        model.eval()
        with torch.no_grad():
            predictions = model(torch.FloatTensor(all_inputs).unsqueeze(0))
            np_predictions = predictions.squeeze(0).numpy()
            val_loss = criterion(predictions, torch.FloatTensor(all_targets).unsqueeze(0))  # Вычисление потерь
        print(f'Epoch [{epoch+1}/{num_epochs}], Total train loss: {total_loss:.4f}, Total val loss: {val_loss:.4f}')

        plt.plot(all_targets[:, 0], all_targets[:, 1], 'g-', label='Истинный путь')
        plt.plot(np_predictions[:, 0], np_predictions[:, 1], 'r-', label='Прогнозируемый путь')
        plt.legend()
        plt.show()

# Заключение
В этом руководстве мы реализовали классический LTC LNN, как описано в оригинальной статье. 

В архитектуре LNN доступны также и другие улучшения:
* [Closed-form Continuous-Time Network models](https://arxiv.org/abs/2106.13898)
* [LTC-SE](https://arxiv.org/abs/2304.08691)
* [Liquid-S4](https://arxiv.org/abs/2209.12951)